# Andrew's code to get the device to connect to the IOTC
This code is based on the device and includes the basics of what Andrew is providing for the translation from the device to what I need for IOTC.

In [1]:
import base64
import json
import requests

In [2]:
def powerStateToString(state):
    return { 
        0 : 'OFF',
        1 : 'ON',
        2 : 'ERROR',
        3 : 'Reserved' 
    }[state]

In [3]:
def pumpStateToString(state):
    return { 
        0 :' ABSENT',
        1 : 'PRESENT',
        2 : 'ERROR',
        3 : 'Reserved' 
    }[state]

In [4]:
# Create a FluidDevice class
class FluidDevice:
    def __init__(self, epochTime, powerState, prePumpFluidState, postPumpFluidState, fluidReservoirLevel, fluidFlowRate, batteryLevel):
        self.epochTime = epochTime
        self.powerState = powerState
        self.prePumpFluidState = prePumpFluidState
        self.postPumpFluidState = postPumpFluidState
        self.fluidReservoirLevel = fluidReservoirLevel
        self.fluidFlowRate = fluidFlowRate
        self.battteryLevel = batteryLevel


In [5]:
def extractPumpData(data_from_message, FluidDevice):
    my_bytes = bytearray(base64.b64decode(data_from_message))
    # Parse out the data bytes into human readable format
    FluidDevice.epochTime = (my_bytes[0] << 24) + (my_bytes[1] << 16) + (my_bytes[2] << 8) + my_bytes[3]
    FluidDevice.powerState = my_bytes[4] & 0x03
    FluidDevice.prePumpFluidState = (my_bytes[4] >> 2) & 0x03
    FluidDevice.postPumpFluidState = (my_bytes[4] >> 4) & 0x03
    FluidDevice.fluidReservoirLevel = my_bytes[5]
    FluidDevice.fluidFlowRate = float(str(my_bytes[6]) + "." + str(my_bytes[7]))
    FluidDevice.batteryLevel = my_bytes[8]
    return FluidDevice


In [6]:
def createIdentifier(device_id):
    my_bytes = bytearray(base64.b64decode(device_id))
    bytesAsString = my_bytes.hex(' ')
    # remove all spaces from bytesAsString
    return bytesAsString.replace(' ', '')

# The following is the Bridge code for IoT Central
The following code is what we need to take the request from the device and send it to IoT Central.

In [7]:
iotcJsonObject = json.loads('{"device":{"deviceId":""},"measurements":{"applicationID":"","applicationName":"","deviceName":"","devEUI":"","adr":false,"dr":0,"fCnt":0,"fPort":0,"data":"","objectJSON":"","confirmedUpLink":false,"devAddr":"","publishedAt":"","PowerState":"","PrePumpFluidState":"","PostPumpFluidState":"","FluidReservoirLevel":"","FluidFlowRate":0,"BatteryLevel":""}}')


# The following is the code for the device

In [8]:
# loading test data from an actual message. This represents the body of the request message
jsonObject = json.loads('{"applicationID":"1","applicationName":"SIP-FieldTrial","deviceName":"xDot-DevBoard-AJC","devEUI":"AIAAAAQBlwY=","rxInfo":[],"txInfo":{"frequency":902900000,"modulation":"LORA","loRaModulationInfo":{"bandwidth":125,"spreadingFactor":9,"codeRate":"4/5","polarizationInversion":false}},"adr":true,"dr":1,"fCnt":1956,"fPort":1,"data":"YWHbPBVLAw5S","objectJSON":"","tags":{},"confirmedUplink":true,"devAddr":"AYjTEQ==","publishedAt":"2021-10-09T18:11:10.003751647Z"}')


# Extract the device data from the jsonObject

In [9]:
#extract the data from the message
pumpDevice = extractPumpData(jsonObject['data'], FluidDevice)

## Map the SIP pump data to the iot central object


In [10]:
# create a function that accepts iotcJsonObject, pumpDevice, and jsonObject and maps the data
def mapData(iotcJsonObject, pumpDevice, jsonObject):
    iotcJsonObject['device']['deviceId'] = createIdentifier(jsonObject['devEUI'])
    iotcJsonObject['measurements']['applicationID'] = jsonObject['applicationID']
    iotcJsonObject['measurements']['applicationName'] = jsonObject['applicationName']
    iotcJsonObject['measurements']['deviceName'] = jsonObject['deviceName']
    iotcJsonObject['measurements']['devEUI'] = jsonObject['devEUI']
    iotcJsonObject['measurements']['adr'] = jsonObject['adr']
    iotcJsonObject['measurements']['dr'] = jsonObject['dr']
    iotcJsonObject['measurements']['fCnt'] = jsonObject['fCnt']
    iotcJsonObject['measurements']['fPort'] = jsonObject['fPort']
    iotcJsonObject['measurements']['data'] = jsonObject['data']
    iotcJsonObject['measurements']['objectJSON'] = jsonObject['objectJSON']
    iotcJsonObject['measurements']['confirmedUpLink'] = jsonObject['confirmedUplink']
    iotcJsonObject['measurements']['devAddr'] = jsonObject['devAddr']
    iotcJsonObject['measurements']['publishedAt'] = jsonObject['publishedAt']
    iotcJsonObject['measurements']['PowerState'] = pumpDevice.powerState
    iotcJsonObject['measurements']['PrePumpFluidState'] = pumpDevice.prePumpFluidState
    iotcJsonObject['measurements']['PostPumpFluidState'] = pumpDevice.postPumpFluidState
    iotcJsonObject['measurements']['FluidReservoirLevel'] = pumpDevice.fluidReservoirLevel
    iotcJsonObject['measurements']['FluidFlowRate'] = pumpDevice.fluidFlowRate
    iotcJsonObject['measurements']['BatteryLevel'] = pumpDevice.batteryLevel
    return iotcJsonObject


In [18]:
# post to IoT Central device bridge
# post to https://iotc-fnjvlbniucwg5pg.azurewebsites.net/api/IoTCIntegration?code=3uW4AC5w/Kksna1kwk6g67qR26lWfew4va2k12eCwMLxDbdkjcaa8A==
url = "http://localhost:7071/api/HttpTriggerPumpDeviceToBridge"
x = requests.post(url, json=iotcJsonObject)
print(x.text)



# Creating the HTTP Trigger Function for the device
In this next section we want to help create the HTTP Trigger function for the device. For this we need to pass the body value taken from the Chirpstack HTPP hook and build a module in the function that does what this notebook does until we have the above body created for the IoTC Bridge

In [20]:
testUrl = "http://localhost:7071/api/HttpTriggerPumpDeviceToBridge"
testChirpStackBody = json.loads('{"applicationID":"1","applicationName":"SIP-FieldTrial","deviceName":"xDot-DevBoard-AJC","devEUI":"AIAAAAQBlwY=","rxInfo":[],"txInfo":{"frequency":902900000,"modulation":"LORA","loRaModulationInfo":{"bandwidth":125,"spreadingFactor":9,"codeRate":"4/5","polarizationInversion":false}},"adr":true,"dr":1,"fCnt":1956,"fPort":1,"data":"YWHbPBVLAw5S","objectJSON":"","tags":{},"confirmedUplink":true,"devAddr":"AYjTEQ==","publishedAt":"2021-10-09T18:11:10.003751647Z"}')
x2 = requests.post(testUrl, json=testChirpStackBody)
print(x2.text)

<Response [200]>
